## 制作基本预测脚本

如果您正在遵循目录结构，那么现在应该打开 model/Train.py 文件。你先要加载虹膜数据集，并使用一个简单的决策树分类器来训练模型。训练完成后，我将使用 joblib 库保存模型，并将精度分数报告给用户。

In [1]:
import os
os.chdir(r'D:\soft_code\machine_learning\machinelearning\ML_example_cv_gridsearch')

In [ ]:
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib

def train_model():
   iris_df = datasets.load_iris()

   x = iris_df.data
   y = iris_df.target

   X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
   dt = DecisionTreeClassifier().fit(X_train, y_train)
   preds = dt.predict(X_test)

   accuracy = accuracy_score(y_test, preds)
   joblib.dump(dt, 'model/iris-model.model')
   print('Model Training Finished.\n\tAccuracy obtained: {}'.format(accuracy))

## 部署

现在你可以打开 app.py 文件并执行一些导入操作。你需要操作系统模块：Flask 和 Flask RESTful 中的一些东西，它们是 10 秒前创建的模型训练脚本，你还要将它们和 joblib 加载到训练模型中：

In [ ]:
import os
from flask import Flask, jsonify, request
from flask_restful import Api, Resource
from model.Train import train_model
from sklearn.externals import joblib

app = Flask(__name__)
api = Api(app)

if not os.path.isfile('model/iris-model.model'):
   train_model()

model = joblib.load('model/iris-model.model')

class MakePrediction(Resource):
   @staticmethod
   def post():
       posted_data = request.get_json()
       sepal_length = posted_data['sepal_length']
       sepal_width = posted_data['sepal_width']
       petal_length = posted_data['petal_length']
       petal_width = posted_data['petal_width']

       prediction = model.predict([[sepal_length, sepal_width, petal_length, petal_width]])[0]
       if prediction == 0:
           predicted_class = 'Iris-setosa'
       elif prediction == 1:
           predicted_class = 'Iris-versicolor'
       else:
           predicted_class = 'Iris-virginica'

       return jsonify({
           'Prediction': predicted_class
       })

api.add_resource(MakePrediction, '/predict')

if __name__ == '__main__':
   app.run(debug=True)